In [1]:
!pip install num2words
!pip install 'nltk==3.5'

     |████████████████████████████████| 101 kB 4.1 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=ffb05a78c6e147ece8b4646c5eba9621478cf59cc49ac182ac7ce604e1587e02
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
     |████████████████████████████████| 1.4 MB 4.5 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=ca66cdc0858c68c88660f52d8c4aedbb9d4168ec8a9d73fa3fcf3b470109f716
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pre

In [2]:
import numpy as np
import pandas as pd 
import re
from num2words import num2words 
from tqdm import tqdm 
import gensim
from gensim.models import KeyedVectors
import pickle
from nltk.translate.bleu_score import sentence_bleu
import matplotlib.pyplot as plt
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.applications import InceptionV3,Xception
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers.merge import add
from tensorflow.keras.layers import GlobalAveragePooling1D,LayerNormalization,Bidirectional,LSTM,Dense,RepeatVector,Embedding,Dropout,Activation,Input,BatchNormalization,GlobalAveragePooling2D,Flatten,Layer,concatenate,Lambda,Add,dot,MaxPooling1D,Conv1D,Multiply,GRU
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,SGD,Adadelta
import tensorflow as tf


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
cap=pd.read_csv('../input/flickr-image-dataset/flickr30k_images/results.csv',sep='|')
cap

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
...,...,...,...
158910,998845445.jpg,0,A man in shorts and a Hawaiian shirt leans ov...
158911,998845445.jpg,1,"A young man hanging over the side of a boat ,..."
158912,998845445.jpg,2,A man is leaning off of the side of a blue an...
158913,998845445.jpg,3,"A man riding a small boat in a harbor , with ..."


In [4]:
# cap=cap[:100]
# cap
cap.columns=['image_name', 'comment_number', 'comment']

In [5]:
cap.isna().sum()

cap=cap.drop(cap[cap.image_name=='2199200615.jpg'].index)
cap[cap.comment.isna()]

,image_name,comment_number,comment


In [6]:
def preprocessing(text):
    pattern='https?:\S+|[^a-zA-Z0-9]+'
    text=re.sub(pattern,' ',text.lower()).strip()
    text='startseq '+text+' endseq'
    tokens=[]
    for token in text.split():
        try:
            token=num2words(token)
        except:
            pass

        tokens.append(token)
    return " ".join(tokens)


In [7]:
cap.comment=cap.comment.apply(preprocessing)

In [8]:
cap.comment.str.split().apply(len).quantile(0.99)

32.0

In [9]:
language=list(cap.comment)
tokenizer=Tokenizer()
tokenizer.fit_on_texts(language)

In [10]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

18131

In [11]:
# import pickle

# # saving
# with open('./tokenizer.pickle', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
# c=cap[['comment','image_name']].to_dict('dict')
captions=dict()
# n=len(c['comment'])
# print(n)
img=list(cap.image_name)
com=list(cap.comment)
n=len(img)
for i in range(n):
    # print(i)
    k=img[i]
    v=com[i]
    if k not in captions:
        captions[k]=[]
    captions[k].append(v)


In [13]:
# captions

In [14]:
img=list(cap.image_name)

In [15]:
# img

In [16]:
unique_imgs=list(set(img))
len(unique_imgs)



31782

In [17]:
# X_img={}
# for i in tqdm(unique_imgs):
#     if i not in X_img:
#         img_path='../input/flickr-image-dataset/flickr30k_images/flickr30k_images/'+str(i)
#         im = load_img(img_path, target_size=(224,224,3))
#         im = img_to_array(im)
#         im=preprocess_input(im)
#         # im=im/255.
#         X_img[i]=im
# np.savez('/content/drive/MyDrive/Image Captioning/inc_img.npz',**X_img)

# X_img=np.load('/content/drive/MyDrive/Image Captioning/inc_img.npz')
# X_img=dict(X_img)

In [18]:
val_imgs=unique_imgs[30000:]
# X_img_val={}
val_captions={}
for i in val_imgs:
#     X_img_val[i]=X_img[i]
    val_captions[i]=captions[i]
#     X_img.pop(i)
    captions.pop(i)    



In [19]:
def data_generator(captions, tokenizer, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while True:
        image_ids=list(captions.keys())
        for i in image_ids:
            n+=1
            img_path='../input/flickr-image-dataset/flickr30k_images/flickr30k_images/'+str(i)
            im = load_img(img_path, target_size=(224,224,3))
            im = img_to_array(im)
            im=preprocess_input(im)
            image=im
            caption_list=captions[i]
            for c in caption_list:
                seq = tokenizer.texts_to_sequences([c])[0]
                
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(image)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                # print('te')
                yield [np.array(X1), np.array(X2)], np.array(y)
                X1, X2, y = list(), list(), list()
                n=0

In [20]:
class MultiHeadAttention(Layer):
    def __init__(self,embdim,nheads):
        super(MultiHeadAttention,self).__init__()
        self.embdim=embdim
        self.nheads=nheads
        self.wq=Dense(embdim)
        self.wk=Dense(embdim)
        self.wv=Dense(embdim)
        self.out_dense=Dense(embdim)
        self.depth=embdim//nheads
 
    def split_heads(self,x,batch_size):
        x=tf.reshape(x,(batch_size,-1,self.nheads,self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
 
    def scaled_dot_attention(self,q,k,v):
        qk=tf.matmul(q,k,transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_qk=qk/tf.math.sqrt(dk)
        attention_weights=tf.nn.softmax(scaled_qk,axis=-1)
        output=tf.matmul(attention_weights,v)
        return output,attention_weights



    def call(self,inputs):
        q=self.wq(inputs)
        v=self.wv(inputs)
        k=self.wk(inputs)
        batch_size=tf.shape(inputs)[0]
        q=self.split_heads(q,batch_size)
        k=self.split_heads(k,batch_size)
        v=self.split_heads(v,batch_size)

        scaled_attention,weights=self.scaled_dot_attention(q,k,v)
        scaled_attention= tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention=tf.reshape(scaled_attention,(batch_size,-1,self.embdim))
        output=self.out_dense(concat_attention)
        return output

    
class PositionalEncoding(Layer):
  
    def __init__(self, d_model):
        super(PositionalEncoding,self).__init__()
        self.d_model = d_model
 
    def call(self, inputs):
        position = tf.shape(inputs)[1]
        position_dims = tf.range(position)[:, tf.newaxis]
        embed_dims = tf.range(self.d_model)[tf.newaxis, :]
        angle_rates = 1 / tf.pow(
            10000.0, tf.cast(
                (2 * (embed_dims // 2)) / self.d_model, tf.float32))
        angle_rads = tf.cast(position_dims, tf.float32) * angle_rates

        sines = tf.sin(angle_rads[:, 0::2])
        cosines = tf.cos(angle_rads[:, 1::2]) 
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)
 
 
 
class Transformer(Layer):
    def __init__(self,embdim,nheads,dff,rate,**kwargs):
        super(Transformer,self).__init__()
        self.ffn1=Dense(dff, activation='relu')
        self.ffn3=Dense(dff/2,activation='relu')
        self.ffn4=Dense(dff/4,activation='relu')
        self.ffn2=Dense(embdim)
        self.mha=MultiHeadAttention(embdim,nheads)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.dropout3 = Dropout(rate)
        self.dropout4 = Dropout(rate)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'embdim': embdim,
            'nheads': nheads,
            'dff': dff,
            'rate': rate,
        })
        return config  
    
    def call(self,inputs):
        att_out=self.mha(inputs)
        att_out=self.dropout1(att_out)
        out1=self.layernorm1(inputs+att_out)
        ff=self.ffn1(out1)
        ff=self.ffn2(ff)
        ff=self.dropout2(ff)
        return self.layernorm2(out1+ff)




class attention(Layer):
    def __init__(self,dim):
        super(attention,self).__init__()
        self.W=Dense(dim)
        self.W2=Dense(dim)
        self.W3=Dense(1)

    def call(self,features,prev):
        prev=tf.expand_dims(prev, 1)
        prev=self.W2(prev)
        c=self.W(features)
        c=tf.math.tanh(c+prev)
        score=self.W3(c)
        # print(c.shape)
        aw=tf.nn.softmax(score,axis=1)   
        q=aw*features
        q=tf.reduce_sum(q,axis=1)
        return q


In [21]:
image_encoder=InceptionV3(weights='imagenet',include_top=False)
image_encoder.summary()

87916544/87910968 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________

In [22]:
for layer in image_encoder.layers:
        layer.trainable = False

In [23]:
max_len=25
word2vec = KeyedVectors.load_word2vec_format('../input/crawl300d2msubword/crawl-300d-2M.vec', binary=False)
embedding_matrix=np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in word2vec.key_to_index:
        embedding_matrix[i]=word2vec.word_vec(word)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  


In [24]:
X_i=image_encoder.output
X_i = GlobalAveragePooling2D()(X_i)
X_i= Dense(512, activation='relu')(X_i)
X_i=RepeatVector(max_len)(X_i)

embdim=300
nheads=10
dff=512
rate=0.2
embedding_layer=Embedding(vocab_size,300,weights=[embedding_matrix],input_length=max_len,trainable=False)
transform=Transformer(embdim,nheads,dff,rate)

X_cap_i=Input(shape=(max_len,))
X_cap=embedding_layer(X_cap_i)
X_cap=LSTM(256, return_sequences=True)(X_cap)
# X_cap=Flatten()(X_cap)
# X_cap=Dense(256,activation='relu')(X_cap)
dec = concatenate([X_i, X_cap])
# dec = Dense(256, activation='relu')(dec)
dec=Dropout(0.5)(dec)
dec=Bidirectional(LSTM(256, return_sequences=False))(dec)
out = Dense(vocab_size, activation='softmax')(dec)
model = Model(inputs=[image_encoder.input, X_cap_i], outputs=out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [25]:
# lr_decay = (1./0.9 -1)/bs
opt = Adam(lr=(1e-4),beta_1=0.9,beta_2=0.999)
loss=tf.keras.losses.CategoricalCrossentropy()
callbacks = [ModelCheckpoint('./image_cap_l.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1),
             EarlyStopping(monitor='val_loss', patience=4, verbose=1, min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, cooldown=0, min_lr=1e-7, verbose=1)]

model.load_weights('../input/image-cap-l/image_cap.h5')
model.compile(optimizer=opt, loss=loss ,metrics=['accuracy'])

In [26]:
number_pics_per_batch = 5
steps = len(captions)//number_pics_per_batch
val_steps=len(val_captions)//number_pics_per_batch
train_generator=data_generator(captions,tokenizer,max_len,number_pics_per_batch)
val_generator=data_generator(val_captions,tokenizer,max_len,number_pics_per_batch)
model.fit(train_generator,validation_data=val_generator,epochs=5,steps_per_epoch=steps,validation_steps=val_steps,callbacks=callbacks)

Epoch 1/5
6000/6000 [==============================] - 3328s 553ms/step - loss: 2.9202 - accuracy: 0.3961 - val_loss: 2.9768 - val_accuracy: 0.3916

Epoch 00001: val_loss improved from inf to 2.97679, saving model to ./image_cap_l.h5
Epoch 2/5
6000/6000 [==============================] - 2864s 477ms/step - loss: 2.8479 - accuracy: 0.4009 - val_loss: 2.9807 - val_accuracy: 0.3915

Epoch 00002: val_loss did not improve from 2.97679
Epoch 3/5
6000/6000 [==============================] - 2859s 476ms/step - loss: 2.7957 - accuracy: 0.4054 - val_loss: 2.9828 - val_accuracy: 0.3919

Epoch 00003: val_loss did not improve from 2.97679

Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.9999999242136255e-05.
Epoch 4/5
6000/6000 [==============================] - 2873s 479ms/step - loss: 2.7455 - accuracy: 0.4105 - val_loss: 2.9647 - val_accuracy: 0.3942

Epoch 00004: val_loss improved from 2.97679 to 2.96465, saving model to ./image_cap_l.h5
Epoch 5/5
6000/6000 [=========================

In [27]:
# model.save("./image_cap_model_flickr40k.h5")


In [28]:
# new_model = tf.keras.models.load_model('./image_cap_model_flickr40k.h5', custom_objects={'Transformer': Transformer})
# # Show the model architecture
# new_model.summary()

In [29]:
index_word = dict([(index,word) for word, index in tokenizer.word_index.items()])

def greedySearch(img_path):
    in_text = 'startseq'
    img_path='../input/flickr-image-dataset/flickr30k_images/flickr30k_images/'+str(img_path)
    im = load_img(img_path, target_size=(224,224,3))
    im = img_to_array(im)
    photo=preprocess_input(im)
    photo=photo.reshape((1,224,224,3))
    for i in range(max_len):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence],max_len)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = index_word[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [30]:
b1=0
b4=0
for i in tqdm(val_imgs):
    vc=val_captions[i]
    v=[j.split()[1:-1] for j in vc]
    # print(v)
    pc=greedySearch(i).split()
    b4+=sentence_bleu(v,pc)
    b1+=sentence_bleu(v, pc, weights=(1, 0, 0, 0))

b1/=(len(val_imgs))
b4/=(len(val_imgs))
print(b1)
print(b4)

  0%|          | 0/1782 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  0%|          | 8/1782 [00:08<23:27,  1.26it/s]/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it con

0.5913790824532144
0.09944556757551964


In [31]:
m=0

for i in tqdm(val_imgs):
    vc=val_captions[i]
    v=[j[9:-7] for j in vc]
    pc=greedySearch(i)
    m+=nltk.translate.meteor_score.meteor_score(v,pc)

m/=(len(val_imgs))
print(m)


100%|██████████| 1782/1782 [19:20<00:00,  1.54it/s]

0.3659297851778719


In [32]:
val_imgs

['529101401.jpg',
 '2978540629.jpg',
 '4812955659.jpg',
 '3349963125.jpg',
 '2771980357.jpg',
 '201108031.jpg',
 '408904281.jpg',
 '152500619.jpg',
 '3994519104.jpg',
 '2460188548.jpg',
 '492358389.jpg',
 '4971580944.jpg',
 '358012011.jpg',
 '4543568583.jpg',
 '141126420.jpg',
 '38319365.jpg',
 '3168306969.jpg',
 '4898769564.jpg',
 '279233347.jpg',
 '6064872574.jpg',
 '6878448774.jpg',
 '3478591390.jpg',
 '4575727965.jpg',
 '3111482098.jpg',
 '506367606.jpg',
 '6309782007.jpg',
 '3030953639.jpg',
 '525434560.jpg',
 '215214751.jpg',
 '3410629379.jpg',
 '5897297135.jpg',
 '3798453280.jpg',
 '686941225.jpg',
 '2463403123.jpg',
 '496110746.jpg',
 '3853594376.jpg',
 '3155279929.jpg',
 '3300679815.jpg',
 '2222944983.jpg',
 '406308903.jpg',
 '1572286502.jpg',
 '4858171330.jpg',
 '4004004911.jpg',
 '3458396026.jpg',
 '5416571561.jpg',
 '2494259543.jpg',
 '2446846666.jpg',
 '3380072636.jpg',
 '7034076961.jpg',
 '4803984649.jpg',
 '73636276.jpg',
 '4963720718.jpg',
 '295344517.jpg',
 '4429301480

In [33]:
tf.__version__

'2.4.1'